In [1]:
using CSV
using DataFrames
using Z3
using JuMP
using HiGHS

In [ ]:
# model = Model(HiGHS.Optimizer)
# @variable(model, x[1:length(sv)], Bin);
# @constraint(model, c[k=1:length(is)], sum([x[i] for i = is[k]]) >= 1);
# @objective(model, Min, sum(x));
# optimize!(model)
# println("Optimal value: ", objective_value(model))

In [2]:
model = JuMP.Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [51]:
@variable(model, x[1:5,1:10,1:3], Bin)

5×10×3 Array{VariableRef, 3}:
[:, :, 1] =
 x[1,1,1]  x[1,2,1]  x[1,3,1]  x[1,4,1]  …  x[1,8,1]  x[1,9,1]  x[1,10,1]
 x[2,1,1]  x[2,2,1]  x[2,3,1]  x[2,4,1]     x[2,8,1]  x[2,9,1]  x[2,10,1]
 x[3,1,1]  x[3,2,1]  x[3,3,1]  x[3,4,1]     x[3,8,1]  x[3,9,1]  x[3,10,1]
 x[4,1,1]  x[4,2,1]  x[4,3,1]  x[4,4,1]     x[4,8,1]  x[4,9,1]  x[4,10,1]
 x[5,1,1]  x[5,2,1]  x[5,3,1]  x[5,4,1]     x[5,8,1]  x[5,9,1]  x[5,10,1]

[:, :, 2] =
 x[1,1,2]  x[1,2,2]  x[1,3,2]  x[1,4,2]  …  x[1,8,2]  x[1,9,2]  x[1,10,2]
 x[2,1,2]  x[2,2,2]  x[2,3,2]  x[2,4,2]     x[2,8,2]  x[2,9,2]  x[2,10,2]
 x[3,1,2]  x[3,2,2]  x[3,3,2]  x[3,4,2]     x[3,8,2]  x[3,9,2]  x[3,10,2]
 x[4,1,2]  x[4,2,2]  x[4,3,2]  x[4,4,2]     x[4,8,2]  x[4,9,2]  x[4,10,2]
 x[5,1,2]  x[5,2,2]  x[5,3,2]  x[5,4,2]     x[5,8,2]  x[5,9,2]  x[5,10,2]

[:, :, 3] =
 x[1,1,3]  x[1,2,3]  x[1,3,3]  x[1,4,3]  …  x[1,8,3]  x[1,9,3]  x[1,10,3]
 x[2,1,3]  x[2,2,3]  x[2,3,3]  x[2,4,3]     x[2,8,3]  x[2,9,3]  x[2,10,3]
 x[3,1,3]  x[3,2,3]  x[3,3,3]  x[3,4,3]     

In [54]:
maximum.([sum(x[i,:,:]) for i = 1:5])

5-element Vector{AffExpr}:
 x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,5,1] + x[1,6,1] + x[1,7,1] + x[1,8,1] + x[1,9,1] + x[1,10,1] + x[1,1,2] + x[1,2,2] + x[1,3,2] + x[1,4,2] + x[1,5,2] + x[1,6,2] + x[1,7,2] + x[1,8,2] + x[1,9,2] + x[1,10,2] + x[1,1,3] + x[1,2,3] + x[1,3,3] + x[1,4,3] + x[1,5,3] + x[1,6,3] + x[1,7,3] + x[1,8,3] + x[1,9,3] + x[1,10,3]
 x[2,1,1] + x[2,2,1] + x[2,3,1] + x[2,4,1] + x[2,5,1] + x[2,6,1] + x[2,7,1] + x[2,8,1] + x[2,9,1] + x[2,10,1] + x[2,1,2] + x[2,2,2] + x[2,3,2] + x[2,4,2] + x[2,5,2] + x[2,6,2] + x[2,7,2] + x[2,8,2] + x[2,9,2] + x[2,10,2] + x[2,1,3] + x[2,2,3] + x[2,3,3] + x[2,4,3] + x[2,5,3] + x[2,6,3] + x[2,7,3] + x[2,8,3] + x[2,9,3] + x[2,10,3]
 x[3,1,1] + x[3,2,1] + x[3,3,1] + x[3,4,1] + x[3,5,1] + x[3,6,1] + x[3,7,1] + x[3,8,1] + x[3,9,1] + x[3,10,1] + x[3,1,2] + x[3,2,2] + x[3,3,2] + x[3,4,2] + x[3,5,2] + x[3,6,2] + x[3,7,2] + x[3,8,2] + x[3,9,2] + x[3,10,2] + x[3,1,3] + x[3,2,3] + x[3,3,3] + x[3,4,3] + x[3,5,3] + x[3,6,3] + x[3,7,3] + x[3,8,3] 

In [49]:
nslot = 39 # 時間帯割り当て
narea = 10 # エリア数
requirenum = 1 # 必要人数
totaltime = 18 # 最大割り当て時間（スロット数）
totalswitch = 2 # 最大切り替え数
csvfile = "heartful.csv"

model = JuMP.Model(HiGHS.Optimizer)

df = CSV.read(csvfile, DataFrame)
nms = [(i,x) for (i,x) = enumerate(names(df)) if x != "start" && x != "end" && x != "学年" && x != "合計"] # 名前取得
println(nms)

y = zeros(Int, length(nms), nslot)
for (i,(k,nm)) = enumerate(nms)
    for t = 1:nslot
        if ismissing(df[t,k])
            y[i,t] = 1 # 無記入は「可能」
        else
            y[i,t] = df[t,k]
        end
    end
end

@variable(model, x[1:size(y)[1],1:nslot,1:narea], Bin)

for i = 1:size(y)[1]
    for t = 1:nslot
        tmp = @constraint(model, sum(x[i,t,:]) <= y[i,t]) # 可能時間帯に高々一つの割り当て
    end
end

for t = 1:nslot
    for k = 1:narea
        @constraint(model, sum(x[:,t,k]) >= requirenum) # 各時間帯で必要人数確保
    end
end

for i = 1:size(y)[1]
    @constraint(model, sum(x[i,:,:]) <= totaltime) # 労働時間制限
end

# op = []
# for i = 1:size(y)[1]
#     for k = 1:narea
#         tmp = x[i,1,k]
#         for t = 2:nslot
#             tmp += (x[i,t,k] - x[i,t-1,k]) * (x[i,t,k] - x[i,t-1,k])
#         end
#         push!(op, tmp)
#         # @constraint(model, tmp <= totalswitch)
#     end
# end

# for (nm,xs) = area
#     for k = 1:narea
#         tmp = Z3.int_val(ctx, 0)
#         for t = 1:nslot
#             if t == 1
#                 tmp += xs[t][k]
#             else
#                 tmp += abs(xs[t][k] - xs[t-1][k])
#             end
#         end
#         push!(cons, tmp <= totalswitch) # 場所替え数制約
#     end
# end

# @objective(model, Min, sum(x));
@objective(model, Min, op);


[(4, "新井"), (5, "岡村"), (6, "梅岡"), (7, "山代"), (8, "西田"), (9, "佐伯"), (10, "三好"), (11, "森岡"), (12, "森川"), (13, "中村"), (14, "田中"), (15, "西本"), (16, "小野川"), (17, "井上"), (18, "八幡"), (19, "太田垣"), (20, "溝口"), (21, "藤井"), (22, "樋口"), (23, "山田"), (24, "津田"), (25, "山神"), (26, "中野"), (27, "小西"), (28, "又吉"), (29, "吉岡")]


ErrorException: The objective function `Any[2 x[1,2,1]² - 2 x[1,1,1]*x[1,2,1] + x[1,1,1]² + 2 x[1,3,1]² - 2 x[1,2,1]*x[1,3,1] + 2 x[1,4,1]² - 2 x[1,3,1]*x[1,4,1] + 2 x[1,5,1]² - 2 x[1,4,1]*x[1,5,1] + 2 x[1,6,1]² - 2 x[1,5,1]*x[1,6,1] + 2 x[1,7,1]² - 2 x[1,6,1]*x[1,7,1] + 2 x[1,8,1]² - 2 x[1,7,1]*x[1,8,1] + 2 x[1,9,1]² - 2 x[1,8,1]*x[1,9,1] + 2 x[1,10,1]² - 2 x[1,9,1]*x[1,10,1] + 2 x[1,11,1]² - 2 x[1,10,1]*x[1,11,1] + 2 x[1,12,1]² - 2 x[1,11,1]*x[1,12,1] + 2 x[1,13,1]² - 2 x[1,12,1]*x[1,13,1] + 2 x[1,14,1]² - 2 x[1,13,1]*x[1,14,1] + 2 x[1,15,1]² - 2 x[1,14,1]*x[1,15,1] + 2 x[1,16,1]² - 2 x[1,15,1]*x[1,16,1] + 2 x[1,17,1]² - 2 x[1,16,1]*x[1,17,1] + 2 x[1,18,1]² - 2 x[1,17,1]*x[1,18,1] + 2 x[1,19,1]² - 2 x[1,18,1]*x[1,19,1] + 2 x[1,20,1]² - 2 x[1,19,1]*x[1,20,1] + 2 x[1,21,1]² - 2 x[1,20,1]*x[1,21,1] + 2 x[1,22,1]² - 2 x[1,21,1]*x[1,22,1] + 2 x[1,23,1]² - 2 x[1,22,1]*x[1,23,1] + 2 x[1,24,1]² - 2 x[1,23,1]*x[1,24,1] + 2 x[1,25,1]² - 2 x[1,24,1]*x[1,25,1] + 2 x[1,26,1]² - 2 x[1,25,1]*x[1,26

In [50]:
optimize!(model)
println("Optimal value: ", objective_value(model))

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1137 rows, 7220 cols, 21660 nonzeros
1137 rows, 7220 cols, 21660 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   1137 rows
   7220 cols (7220 binary, 0 integer, 0 implied int., 0 continuous)
   21660 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.1s

Solving report
  Status            Optimal
  Primal bound      0
  Dual bound        0
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    0 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Tim

In [47]:
r = Int.(JuMP.value.(x))

table = Dict()
for (i,(_,nm)) = enumerate(nms)
    tmp = []
    for t = 1:nslot
        u = r[i,t,:]
        res = findfirst(x->x==1, u)
        if isnothing(res)
            push!(tmp, 0)
        else
            push!(tmp, res)
        end
    end
    table[nm] = tmp
end
DataFrame(table)


Row,三好,中村,中野,井上,佐伯,八幡,又吉,吉岡,太田垣,小西,小野川,山代,山田,山神,岡村,新井,梅岡,森岡,森川,樋口,津田,溝口,田中,藤井,西本,西田
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,0,9,0,0,0,0,0,5,8,0,0,0,10,1,0,7,0,3,0,6,0,0,4,0,0,2
2,3,0,0,6,0,0,0,0,1,0,8,0,2,5,4,0,0,0,0,0,9,0,0,7,0,10
3,0,4,0,0,0,0,0,6,0,0,5,8,9,7,1,2,0,0,0,0,10,0,0,3,0,0
4,0,0,0,5,6,0,0,0,0,0,0,1,0,7,0,9,2,10,0,0,0,0,3,4,0,8
5,0,5,0,10,4,3,0,0,0,0,0,6,0,0,0,8,1,0,0,2,0,9,0,7,0,0
6,0,4,0,8,6,5,7,0,0,0,0,0,0,0,0,1,0,0,0,2,0,9,0,3,0,10
7,0,3,0,9,8,7,2,0,0,0,0,1,0,6,0,0,0,10,0,0,0,5,4,0,0,0
8,0,0,0,9,0,1,4,0,0,0,0,8,0,7,0,6,0,5,0,0,0,10,2,0,0,3
9,0,0,0,9,8,2,5,0,0,0,0,0,0,3,0,10,0,0,0,0,0,6,1,4,0,7


In [ ]:
"""
nslot = 17 # 時間帯割り当て
narea = 2 # エリア数
requirenum = 1 # 必要人数
totaltime = 15 # 最大割り当て時間（スロット数）
totalswitch = 2 # 最大切り替え数
"""

function maketable(csvfile ;nslot = 17, narea = 2, requirenum = 1, totaltime = 15, totalswitch = 2)
    model = Model(HiGHS.Optimizer)

    df = CSV.read(csvfile, DataFrame)
    nms = [(i,x) for (i,x) = enumerate(names(df)) if x != "start" && x != "end"] # 名前取得
    println(nms)

    y = zeros(Int, 1:length(nms), 1:nslot)
    for (i,(k,nm)) = enumerate(nms)
        for t = 1:nslot
            if ismissing(df[t,k])
                y[i,t] = 1
            else
                y[i,t] = df[t,k]
            end
        end
    end

    @variable(model, x[1:length(nms),1:nslot,1:narea], Bin)

    for (i,nm) = nms
        tmparea = []
        for t = 1:nslot
            @constraint(model, c[], sum(x[i,t,:]) <= df[t,i]);
            # x = Z3.int_const(ctx, "var$(i)_$(t)")
            # f = [Z3.int_const(ctx, "area$(k)_$(i-2)_$(t)") for k = 1:narea] # 2 はオフセットフォーマットに合わせる必要あり
            push!(tmparea, f)
            for v = f
                push!(cons, v >= 0) # 割り当て変数非負条件
            end
            push!(cons, sum(f) <= x) # 可能時間帯に高々一つの割り当て
            if ismissing(df[t,i])
                push!(cons, x == 1) # 無記入は「可能」
            else
                push!(cons, x == Int(df[t,i])) # 可能な時間帯の指定
            end
        end
        area[nm] = tmparea
    end

    for t = 1:nslot
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for (_,xs) = area
                tmp += xs[t][k]
            end
            push!(cons, tmp == requirenum) # 各時間帯で必要人数確保
        end
    end

    for (nm,xs) = area
        tmp = Z3.int_val(ctx, 0)
        for x = xs
            tmp += sum(x)
        end
        push!(cons, tmp <= totaltime) # 労働時間制限
    end

    for (nm,xs) = area
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for t = 1:nslot
                if t == 1
                    tmp += xs[t][k]
                else
                    tmp += abs(xs[t][k] - xs[t-1][k])
                end
            end
            push!(cons, tmp <= totalswitch) # 場所替え数制約
        end
    end

    solver = Z3.Solver(ctx)
    for x = cons
        add(solver, x)
    end

    res = Z3.check(solver)
    if res == Z3.sat
        m = Z3.get_model(solver)
        result = Dict(["$k"=>Z3.get_numeral_int(v) for (k,v) = Z3.consts(m)])
    else
        error("No solution")
    end

    table = Dict()
    for (i,nm) = enumerate(keys(area))
        tmp = []
        for t = 1:nslot
            u = [result["area$(k)_$(i)_$(t)"] for k = 1:narea]
            res = findfirst(x->x==1, u)
            if isnothing(res)
                push!(tmp, 0)
            else
                push!(tmp, res)
            end
        end
        table[nm] = tmp
    end
    DataFrame(table)
end

In [2]:
"""
nslot = 17 # 時間帯割り当て
narea = 2 # エリア数
requirenum = 1 # 必要人数
totaltime = 15 # 最大割り当て時間（スロット数）
totalswitch = 2 # 最大切り替え数
"""

function maketable(csvfile ;nslot = 17, narea = 2, requirenum = 1, totaltime = 15, totalswitch = 2)
    ctx = Z3.Context()

    cons = []
    area = Dict()

    df = CSV.read(csvfile, DataFrame)

    nms = [(i,x) for (i,x) = enumerate(names(df)) if x != "start" && x != "end"] # 名前取得
    println(nms)

    for (i,nm) = nms
        tmparea = []
        for t = 1:nslot
            x = Z3.int_const(ctx, "var$(i)_$(t)")
            f = [Z3.int_const(ctx, "area$(k)_$(i-2)_$(t)") for k = 1:narea] # 2 はオフセットフォーマットに合わせる必要あり
            push!(tmparea, f)
            for v = f
                push!(cons, v >= 0) # 割り当て変数非負条件
            end
            push!(cons, sum(f) <= x) # 可能時間帯に高々一つの割り当て
            if ismissing(df[t,i])
                push!(cons, x == 1) # 無記入は「可能」
            else
                push!(cons, x == Int(df[t,i])) # 可能な時間帯の指定
            end
        end
        area[nm] = tmparea
    end

    for t = 1:nslot
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for (_,xs) = area
                tmp += xs[t][k]
            end
            push!(cons, tmp == requirenum) # 各時間帯で必要人数確保
        end
    end

    for (nm,xs) = area
        tmp = Z3.int_val(ctx, 0)
        for x = xs
            tmp += sum(x)
        end
        push!(cons, tmp <= totaltime) # 労働時間制限
    end

    for (nm,xs) = area
        for k = 1:narea
            tmp = Z3.int_val(ctx, 0)
            for t = 1:nslot
                if t == 1
                    tmp += xs[t][k]
                else
                    tmp += abs(xs[t][k] - xs[t-1][k])
                end
            end
            push!(cons, tmp <= totalswitch) # 場所替え数制約
        end
    end

    solver = Z3.Solver(ctx)
    for x = cons
        add(solver, x)
    end

    res = Z3.check(solver)
    if res == Z3.sat
        m = Z3.get_model(solver)
        result = Dict(["$k"=>Z3.get_numeral_int(v) for (k,v) = Z3.consts(m)])
    else
        error("No solution")
    end

    table = Dict()
    for (i,nm) = enumerate(keys(area))
        tmp = []
        for t = 1:nslot
            u = [result["area$(k)_$(i)_$(t)"] for k = 1:narea]
            res = findfirst(x->x==1, u)
            if isnothing(res)
                push!(tmp, 0)
            else
                push!(tmp, res)
            end
        end
        table[nm] = tmp
    end
    DataFrame(table)
end

maketable (generic function with 1 method)

In [7]:
maketable("testdata.csv", nslot = 17, narea = 2, requirenum = 1, totaltime = 13, totalswitch = 2)

[(3, "山田"), (4, "田中"), (5, "鈴木"), (6, "佐藤")]


Row,佐藤,山田,田中,鈴木
,Any,Any,Any,Any
1,0,1,2,0
2,2,1,0,0
3,1,2,0,0
4,1,2,0,0
5,1,0,0,2
6,0,0,1,2
7,0,0,1,2
8,0,0,1,2
9,0,0,1,2


In [20]:
df

UndefVarError: UndefVarError: df not defined

In [21]:
df = CSV.read("testdata.csv", DataFrame)

Row,start,end,山田,田中,鈴木,佐藤
,Time,Time,Missing,Int64,Int64,Int64
1,08:00:00,08:10:00,missing,0,1,1
2,08:10:00,08:20:00,missing,1,1,1
3,08:20:00,08:30:00,missing,1,1,1
4,08:30:00,08:40:00,missing,1,1,1
5,08:40:00,08:50:00,missing,1,1,1
6,08:50:00,09:00:00,missing,0,1,0
7,09:00:00,09:10:00,missing,0,1,0
8,09:10:00,09:20:00,missing,0,1,0
9,09:20:00,09:30:00,missing,0,1,0


In [24]:
df["山田"]

ArgumentError: ArgumentError: syntax df[column] is not supported use df[!, column] instead